In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [3]:
#sigmoid fonk f(x)=1/(1+exp^-x)
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [4]:
def deriv_sigmoid(x):
    return(x*(1-x))

In [5]:
def relu(x):
    return np.maximum(0,x)

In [6]:
def deriv_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0)
    return der

In [7]:
def leaky_relu(x):
    return np.maximum(0.1*x,x)
    

In [8]:
def deriv_leaky_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0.1)
    return der

In [9]:
def initialize(x, neuron_num):
    np.random.seed(12345)
    w1=2*np.random.random((len(x.columns), neuron_num))-1
    w2=2*np.random.random((neuron_num, 1))-1
    bias1=2*np.random.random()-1
    bias2=2*np.random.random()-1
    return w1, w2, bias1, bias2

In [10]:
def scaler(data):

    flag_min=1000000
    flag_max=-1000000
    data= data.to_numpy()
    
    for item in data:
        flag_min=np.minimum(item,flag_min)
        flag_max=np.maximum(item,flag_max)
        
    range_=flag_max-flag_min
    new_data=pd.DataFrame()
    for item in data:
        item=(item-flag_min)/range_
    return data
                  

In [11]:
def forward_prop(w1, w2, bias1, bias2, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    #out1=leaky_relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    #out2=leaky_relu(z2)
    return z1, out1, z2, out2

In [12]:
def back_prop(w1, w2, bias1, bias2, x, y, z1, out1, z2, out2, lr):
    x=x.reshape(13,1)
    err_2=2*(np.subtract(out2.sum(axis=0), y)) # karenin turevi oldugu icin 2*(y_pred-y_act)
    dummy_2=np.array(deriv_relu(z2)).reshape(1,1)
    #dummy_2=np.array(deriv_leaky_relu(z2)).reshape(1,1)
    dc_dw2=lr*np.dot(out1,dummy_2)*err_2
    w2=np.subtract(w2,dc_dw2)
    dc_db2=lr*err_2
    bias2-=dc_db2

    err_1= 2*(np.subtract(out1.sum(axis=0), y))
    dummy_1=np.array(deriv_relu(z1)).reshape(len(z1),1)
    #dummy_1=np.array(deriv_leaky_relu(z1)).reshape(len(z1),1)
    dc_dw1=lr*np.dot(x,dummy_1.T)*err_1
    w1=np.subtract(w1,dc_dw1)
    dc_db1=lr*err_1
    bias1-=dc_db1
    return w1, w2, bias1, bias2

In [13]:
def forward_prop_test(w1, w2, bias1, bias2, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    return out1

In [14]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [15]:
columns=("CRIM","ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV")

In [16]:
ds=pd.read_csv("/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/housing.csv", sep='\s+', engine='python', names=columns)

In [17]:
#ds.head()

In [18]:
#ds.isnull().sum()

In [19]:
#ds.describe(include='all')

In [20]:
target=ds.loc[:,'MEDV'].copy()

In [21]:
data=ds.drop('MEDV', axis=1).copy()

In [22]:
neuron_num=10

In [23]:
w1, w2, bias1, bias2=initialize(data,neuron_num)

In [24]:
data_normalized = ((data - data.mean()) / data.std(ddof=0)).to_numpy()
target=target.to_numpy()

In [25]:
cutoff = int(len(data) * 0.8)
x_train, x_test = data_normalized[:cutoff], data_normalized[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [26]:
epochs=100

In [27]:
lr=0.00001

In [28]:
train_loss_epoch=list()
test_loss_epoch=list()

In [29]:
for e in range(epochs):
    loss_train=0
    loss_test=0
    for i in range(len(y_train)):   
        z1, out1, z2, out2= forward_prop(w1, w2, bias1, bias2, x_train[i])
        w1, w2, bias1, bias2 = back_prop(w1, w2, bias1, bias2, x_train[i], y_train[i], z1, out1, z2, out2, lr)
        loss_train+=np.square(y_train[i]-(out2.sum(axis=0)))
   
    train_loss_epoch.append(loss_train/(len(y_train)))   
    
    for i in range(len(y_test)):
        z_test1, out_test1, z_test2, out_test2=forward_prop(w1, w2, bias1, bias2, x_test[i])
        loss_test+=np.square(y_test[i]-(out_test2.sum(axis=0)))

    test_loss_epoch.append(loss_test/(len(y_test)))

In [37]:
print(train_loss_epoch)

[array([626.77900611]), array([539.95919425]), array([418.60633793]), array([305.59992008]), array([218.48996537]), array([161.01836948]), array([126.96060248]), array([107.36754004]), array([94.8052383]), array([85.33936714]), array([77.19281589]), array([69.86049534]), array([63.29012615]), array([57.51621485]), array([52.52048087]), array([48.24922515]), array([44.6603526]), array([41.67878252]), array([39.21306246]), array([37.1768785]), array([35.4924348]), array([34.09520645]), array([32.9298458]), array([31.95508001]), array([31.13028947]), array([30.42413769]), array([29.81511395]), array([29.27995738]), array([28.80934599]), array([28.39031664]), array([28.01336375]), array([27.66700143]), array([27.34768113]), array([27.05137551]), array([26.77358456]), array([26.51071094]), array([26.26084457]), array([26.0209123]), array([25.78957565]), array([25.56617484]), array([25.34982635]), array([25.1401726]), array([24.93483775]), array([24.73433901]), array([24.53746373]), array([2

In [30]:
#for i in range(len(y_train)):
   # print(y_train[i], output[i].sum(axis=0))
#    loss+=(y_train[i]-(output2[i].sum(axis=0)))

In [31]:
#print(loss/(len(y_train)))

In [32]:
#out_test=list()
#for i in range(len(y_test)): 
    #z_test1, out_test1, z_test2, out_test2=forward_prop(w1, w2, bias1, bias2, x_test[i])
    #out_test.append(out_test2)
#    out_test.append(forward_prop_test(w1, w2, bias1, bias2, x_test[i]))
#    loss_test+=(y_test[i]-(out_test[i].sum(axis=0)))
#print(loss_test/(len(y_test)))

In [33]:
#for i in range(len(y_test)):
#    print(y_test[i], out_test[i].sum(axis=0))

In [34]:
#for i in range(len(y_train)):
#    print(y_train[i], output2[i])